In [1]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'gay',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI'
]

In [2]:
import json
import random
from glob import glob

In [3]:
templates = [
    'tukar ke JSON berdasarkan schema {schema}, teks `{text}`',
    'text `{text}`, convert to JSON using schema {schema}',
    'teks: {text}\n\ntukar ke JSON using schema {schema}',
    'convert to JSON using schema {schema}\n\ntext: {text}',
    '{text}\n\nJSON berdasarkan schema {schema}',
    'JSON berdasarkan schema {schema}\n\n{text}',
]

In [5]:
data = []

with open('/home/husein/ssd3/instructions/function-calls.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        texts = []
        failed = False
        for no, d in enumerate(l['conversations']):

            if d['role'] == 'user':
                t = '<manusia>: '
                if random.random() > 0.5:
                    c = 'content'
                else:
                    c = 'content'
            else:
                if 'functioncall' in d['content']:
                    c = 'content'
                else:
                    if 'functioncall' in l['conversations'][no - 1]['content']:
                        continue
                    c = 'content'
                t = '<bot>: '

            if d[c] is None:
                break

            if 'functioncall' in d[c]:
                d[c] = d[c].split('FUNCTION RESPONSE:')[0].strip()

            if '<functioncall>' in d[c]:
                d[c] = d[c].split('<functioncall>')[1].strip()
                try:
                    d[c] = eval(d[c])
                    d[c] = json.dumps(d[c])
                except:
                    failed = True
                    continue


            n = d[c]
            n = n.strip()

            if len(n) < 3:
                break

            if d['role'] == 'user':
                n = random.choice(templates).format(schema = l['function_call'], text = n)
            t = t + n
            texts.append(t)

        if failed:
            continue

        if not len(texts):
            continue
        while len(texts) and texts[-1].startswith('<manusia>'):
            texts = texts[:-1]
        if not len(texts):
            continue
        if not texts[0].startswith('<manusia>'):
            continue

        data.append({
            'prompt_input': None,
            'input': '\n'.join(texts[:2]),
            'output': None,
        })
            
len(data)

179339

In [6]:
data_complex = []

with open('/home/husein/ssd3/instructions/function-calls-complex.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        texts = []
        failed = False
        for no, d in enumerate(l['conversations']):

            if d['role'] == 'user':
                t = '<manusia>: '
                if random.random() > 0.5:
                    c = 'content'
                else:
                    c = 'content'
            else:
                if 'functioncall' in d['content']:
                    c = 'content'
                else:
                    if 'functioncall' in l['conversations'][no - 1]['content']:
                        continue
                    c = 'content'
                t = '<bot>: '

            if d[c] is None:
                break

            if 'functioncall' in d[c]:
                d[c] = d[c].split('FUNCTION RESPONSE:')[0].strip()

            if '<functioncall>' in d[c]:
                d[c] = d[c].split('<functioncall>')[1].strip()
                try:
                    d[c] = eval(d[c])
                    d[c] = json.dumps(d[c])
                except:
                    failed = True
                    continue


            n = d[c]
            n = n.strip()

            if len(n) < 3:
                break

            if d['role'] == 'user':
                n = random.choice(templates).format(schema = l['function_call'], text = n)
            t = t + n
            texts.append(t)

        if failed:
            continue

        if not len(texts):
            continue
        while len(texts) and texts[-1].startswith('<manusia>'):
            texts = texts[:-1]
        if not len(texts):
            continue
        if not texts[0].startswith('<manusia>'):
            continue

        data_complex.append({
            'prompt_input': None,
            'input': '\n'.join(texts[:2]),
            'output': None,
        })
            
len(data_complex)

24984

In [7]:
def generate_and_tokenize_prompt(row):
    texts = ['<s>']

    if 'function_call' in row:
        t = row['function_call']
        texts.append(f'\n[FUNCTIONCALL]\n{t}\n')

    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human.strip())
            outputs.append(bot.strip())
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    for u, a in zip(inputs, outputs):
        texts.append(f'[INST] {u.strip()} [/INST] {a.strip()}</s> ')

    prompt = ''.join(texts)
    return {'text': prompt}

In [8]:
print(generate_and_tokenize_prompt(data[-4])['text'])

<s>[INST] Hai, saya ingin mengira IMT saya. Saya berat 70 kilogram dan tinggi saya 1.75 meter.

JSON berdasarkan schema {
 "name": "hitung_imt",
 "description": "Hitung Indeks Massa Tubuh (IMT)",
 "parameters": {
 "type": "object",
 "properties": {
 "berat": {
 "type": "number",
 "description": "Berat dalam kilogram"
 },
 "tinggi": {
 "type": "number",
 "description": "Tinggi dalam meter"
 }
 },
 "required": [
 "berat",
 "tinggi"
 ]
 }
} [/INST] {"name": "hitung_imt", "arguments": {"berat": 70, "tinggi": 1.75}}</s> 


In [9]:
print(generate_and_tokenize_prompt(data_complex[-4])['text'])

<s>[INST] saya ada 2 orang kena edit,
1. tambah peranan1, emel test@emel
2. buang pentadbir, emel mike@emel

JSON berdasarkan schema {"name":"KemaskiniPeranan","description":"Fungsi ini digunakan untuk mengemaskini peranan pengguna. Ia menerima pelbagai objek. Setiap objek hendaklah memasukkan nama peranan, alamat emel, dan tindakan yang menentukan sama ada peranan itu perlu ditambah atau dialih keluar.","parameters":{"type":"object","properties":{"pengguna_peranan":{"type":"array","items":{"type":"object","properties":{"nama_peranan":{"type":"string","enum":["peranan1","peranan2","pentadbir"],"description":"Nama peranan yang akan dikemaskini."},"emel":{"type":"string","description":"Alamat emel pengguna yang peranan sedang dikemaskini."},"tindakan":{"type":"string","enum":["tambah","buang"],"description":"Tindakan yang perlu dilakukan. 'tambah' untuk memberikan peranan kepada pengguna, 'buang' untuk mengeluarkan peranan daripada pengguna."}},"required":["nama_peranan","emel","tindakan

In [10]:
from tqdm import tqdm

for d in data:
    generate_and_tokenize_prompt(d)['text']

In [11]:
print(generate_and_tokenize_prompt(data[1])['text'])

<s>[INST] JSON berdasarkan schema {
    "name": "dapatkan_data_pasaran",
    "description": "Dapatkan data harga saham terkini",
    "parameters": {
        "type": "object",
        "properties": {
            "saham": {
                "type": "string",
                "description": "Kod saham untuk mendapatkan data"
            }
        },
        "required": [
            "saham"
        ]
    }
}

Bolehkah anda memberitahu saya data harga saham terkini untuk syarikat Aplikasi Bintang? [/INST] {"name": "dapatkan_data_pasaran", "arguments": {"saham": "Aplikasi Bintang"}}</s> 


In [15]:
import json

with open('prepared-chatgpt-malay-function-call.jsonl', 'w') as fopen:
    for l in data:
        fopen.write(f'{json.dumps(l)}\n')

In [16]:
with open('prepared-chatgpt-malay-function-call-complex.jsonl', 'w') as fopen:
    for l in data_complex:
        fopen.write(f'{json.dumps(l)}\n')